## Part 1: Preprocessing and EDA


### Data Import & Cleaning

In [107]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### Problem 1: Data Import & Cleaning
- convert profit and sales to numeric

In [71]:
orders = pd.read_csv('Orders.csv',index_col='Row.ID',
                     parse_dates=['Order.Date','Ship.Date'])
returns = pd.read_csv('Returns.csv', index_col=0)

In [72]:
# remove dollar sign
cols_to_change = ['Profit', 'Sales']
orders[cols_to_change] = orders[cols_to_change].replace({'\$':''}, regex = True)
orders[cols_to_change] = orders[cols_to_change].replace({'\,':''}, regex = True)

In [73]:
orders['Sales'] = pd.to_numeric(orders['Sales'])
orders['Profit'] = pd.to_numeric(orders['Profit'])

### Problem 2: Inventory Management
- look at seasonal trend
- does the seasonal trend depend on different categories
- Hint: look at the Quantity feature (number of products in an order)

In [ ]:
# ## Trying to graph all data sales by time

# orders[['Sales', 'Order.Date']]
# orders2 = orders.groupby('Order.Date').agg(['sum':'Sales'])
# # orders['Order.Date'] = pd.to_datetime(orders['Order.Date'])
# # orders['Ship.Date'] = pd.to_datetime(orders['Ship.Date'])
# # plt.plot(orders['Order.Date'], orders['Sales'])
# orders2

In [85]:
## Creating a month and year column

orders['month'] = pd.DatetimeIndex(orders['Order.Date']).month
orders['year'] = pd.DatetimeIndex(orders['Order.Date']).year

In [100]:
## groupby month
orders_date_qty = orders.groupby(['year','month']).agg({'Quantity':'sum'})
orders_date_sales = orders.groupby(['year','month']).agg({'Sales':'sum'})

In [113]:
orders_date_qty.reset_index()


,year,month,Quantity
0,2012,1,1463
1,2012,2,1401
2,2012,3,1745
3,2012,4,1934
4,2012,5,2167
5,2012,6,3002
6,2012,7,1836
7,2012,8,3020
8,2012,9,3644
9,2012,10,2981


### Problem 3: Customer Returns
Merge returns and orders using Order.ID
1. lost profits due to returns
2. number of customers that returned more than one
3. regions that return more orders
4. Which categories and sub-categories are more likely to be returned

In [125]:
OR_merge = orders.merge(returns, how='left', left_on='Order.ID', right_on='Order ID')

In [124]:
OR_merge.isna().sum()

Order.ID              0
Order.Date            0
Ship.Date             0
Ship.Mode             0
Customer.ID           0
Customer.Name         0
Segment               0
Postal.Code       41296
City                  0
State                 0
Country               0
Region_x              0
Market                0
Product.ID            0
Category              0
Sub.Category          0
Product.Name          0
Sales                 0
Quantity              0
Discount              0
Profit                0
Shipping.Cost         0
Order.Priority        0
month                 0
year                  0
Order ID          49070
Region_y          49070
dtype: int64

In [127]:
OR_merge.loc[OR_merge['Region_y'].isna(),'return_flag'] = 0
OR_merge.loc[~OR_merge['Region_y'].isna(),'return_flag'] = 1

In [128]:
OR_merge['process_time'] = OR_merge['Ship.Date'] - OR_merge['Order.Date']
OR_merge.groupby(['return_flag']).agg({'process_time':'mean'})

,process_time
return_flag,
0.0,3 days 23:12:20.542082738
1.0,4 days 00:34:22.702702702


In [136]:
orders['Product.Name'].value_counts()

Staples                                                 227
Cardinal Index Tab, Clear                                92
Eldon File Cart, Single Width                            90
Rogers File Cart, Single Width                           84
Ibico Index Tab, Clear                                   83
                                                       ... 
Lexmark X 9575 Professional All-in-One Color Printer      1
NETGEAR RangeMax WNR1000 Wireless Router                  1
4009 Highlighters                                         1
Xerox 1938                                                1
Cisco SPA 501G IP Phone                                   1
Name: Product.Name, Length: 3788, dtype: int64

In [139]:
orders['Product.Name'].nunique()

3788

In [140]:
orders['Order.ID'].nunique()

25728

In [142]:
orders.shape

(51290, 25)